In [1]:
import pandas as pd
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer 
from nltk.stem import SnowballStemmer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from statistics import mean 
import numpy as np
import keras 
from keras.models import Sequential, Model 
from keras import layers
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd
from nltk import pos_tag
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression 
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer
import keras
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta,Adam,RMSprop
from keras.utils import np_utils
from sklearn.metrics import accuracy_score,classification_report
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Bidirectional, GRU, Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sunwoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sunwoo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from __future__ import division, print_function
#from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape,SpatialDropout1D, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string
import keras
import tensorflow as tf
import pandas as pd
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from statistics import mean
import numpy as np
import keras
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd
from nltk import pos_tag
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer
import keras
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta,Adam,RMSprop
from keras.utils import np_utils
from sklearn.metrics import accuracy_score,classification_report
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Bidirectional, GRU, Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
import scipy

In [3]:
def preprocess_paragraph(row):
    text = row['content']
    text= text.lower()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub(' ', text)
    meaningless_words = ['words', 'business wire', 'bwr english', 'copyright', 'businesswire.com',
                         'dow jones newswires','djdn english','dow jones institutional news', 
                         'all rights reserved', 'dow jones company inc', 'pr Newswire',
                         'prn english', 'the wall street journal',  'dow jones & company inc',
                         'j b4 english']
    for words in meaningless_words:
        text = text.replace(words, '')
    
    # picking paragraph containing keywords 
    text = text.split("/n/n")
    text = [para for para in text if "license" in para or "licensing" in para]
    text = ''.join(text)
    
        
    # remove remaining tokens that are not alphabetic
    text = text.split(' ')
    text = [word for word in text if word.isalpha()]
    
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    stemmer = SnowballStemmer("english")
    lemmatizer = WordNetLemmatizer()
    stemmed_text = []
    for word in text:
        #stemmed_text.append(stemmer.stem(word))
        stemmed_text.append(lemmatizer.lemmatize(word))
        
    text = " ".join(stemmed_text)
    row['content'] = text
    return(row)

In [4]:
def preprocess_full_text(row):
    text = row['content']
    text= text.lower()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub(' ', text)
    meaningless_words = ['words', 'business wire', 'bwr english', 'copyright', 'businesswire.com',
                         'dow jones newswires','djdn english','dow jones institutional news', 
                         'all rights reserved', 'dow jones company inc', 'pr Newswire',
                         'prn english', 'the wall street journal',  'dow jones & company inc',
                         'j b4 english']
    for words in meaningless_words:
        text = text.replace(words, '')
        
    # remove remaining tokens that are not alphabetic
    text = text.split(' ')
    text = [word for word in text if word.isalpha()]
    
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    stemmer = SnowballStemmer("english")
    lemmatizer = WordNetLemmatizer()
    stemmed_text = []
    for word in text:
        #stemmed_text.append(stemmer.stem(word))
        stemmed_text.append(lemmatizer.lemmatize(word))
        
    text = " ".join(stemmed_text)
    row['content'] = text
    return(row)

In [5]:
def support_vm(X,y,z= None):
    clf = svm.SVC(kernel='linear')
    kf = KFold(n_splits=10)
    kf.get_n_splits(X) 
    KFold(n_splits=10, random_state=None, shuffle=False)
    accuracy=[]
    precision = []
    recall = []
    f1 = []
    
    temp = 0
    new_clf = svm.SVC(kernel='linear')
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        clf_report = classification_report(y_test, predictions, output_dict = True)
        accuracy.append(clf_report['accuracy'])
        precision.append(clf_report['weighted avg']['precision'])
        recall.append(clf_report['weighted avg']['recall'])
        f1.append(clf_report['weighted avg']['f1-score'])
        f1score = clf_report['weighted avg']['f1-score']
        if f1score > temp:
            new_clf = clf
            temp = f1score
    print("Accuracy: {}, Precision: {}, Recall: {}, F1-score: {}".format(mean(accuracy),mean(precision),mean(recall),mean(f1)))
    if z is not None:
        predictions = new_clf.predict(z)
        return predictions
    else:
        return None

def random_forest(X,y,z =None):
    clf = RandomForestClassifier()
    kf = KFold(n_splits=10)
    kf.get_n_splits(X)
    KFold(n_splits=10, random_state=None, shuffle=False)

    accuracy=[]
    precision = []
    recall = []
    f1 = []
    
    temp = 0
    new_clf = RandomForestClassifier()
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        clf_report = classification_report(y_test, predictions, output_dict = True)
        accuracy.append(clf_report['accuracy'])
        precision.append(clf_report['weighted avg']['precision'])
        f1.append(clf_report['weighted avg']['f1-score'])
        recall.append(clf_report['weighted avg']['recall'])
        f1score = clf_report['weighted avg']['f1-score']
        if f1score > temp:
            new_clf = clf
            temp = f1score
    print("Accuracy: {}, Precision: {}, Recall: {}, F1-score: {}".format(mean(accuracy),mean(precision),mean(recall),mean(f1)))
    if z is not None:
        predictions = new_clf.predict(z)
        return predictions
    else:
        return None
    
def logit(X,y,z =None):
    # Logistics Regression
    clf = LogisticRegression(random_state=0).fit(X, y)
    kf = KFold(n_splits=10)
    kf.get_n_splits(X)
    KFold(n_splits=10, random_state=None, shuffle=False)
    
    accuracy=[]
    precision = []
    recall = []
    f1 = []
    
    temp = 0
    new_clf = LogisticRegression(random_state=0)
    
    for train_index, test_index in kf.split(X):
        #print("Train:", train_index, "Test:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        clf_report = classification_report(y_test, predictions, output_dict = True)
        accuracy.append(clf_report['accuracy'])
        precision.append(clf_report['weighted avg']['precision'])
        recall.append(clf_report['weighted avg']['recall'])
        f1.append(clf_report['weighted avg']['f1-score'])
        f1score = clf_report['weighted avg']['f1-score']
        if f1score > temp:
            new_clf = clf
            temp = f1score
    print("Accuracy: {}, Precision: {}, Recall: {}, F1-score: {}".format(mean(accuracy),mean(precision),mean(recall),mean(f1)))
    if z is not None:
        predictions = new_clf.predict(z)
        return predictions
    else:
        return None

def xgboost(X,y,z =None):
    clf=xgb.XGBClassifier(random_state=1,learning_rate=0.01).fit(X,y)
    kf = KFold(n_splits=10)
    kf.get_n_splits(X)
    KFold(n_splits=10, random_state=None, shuffle=False)

    accuracy=[]
    precision = []
    recall = []
    f1 = []
    
    temp = 0
    new_clf = LogisticRegression(random_state=0)
    
    for train_index, test_index in kf.split(X):
        #print("Train:", train_index, "Test:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        clf_report = classification_report(y_test, predictions, output_dict = True)
        accuracy.append(clf_report['accuracy'])
        precision.append(clf_report['weighted avg']['precision'])
        recall.append(clf_report['weighted avg']['recall'])
        f1.append(clf_report['weighted avg']['f1-score'])
    print("Accuracy: {}, Precision: {}, Recall: {}, F1-score: {}".format(mean(accuracy),mean(precision),mean(recall),mean(f1)))
    if z is not None:
        predictions = new_clf.predict(z)
        return predictions
    else:
        return None
    
def dctree(X,y,z =None):
    clf = DecisionTreeClassifier()
    kf = KFold(n_splits=10)
    kf.get_n_splits(X)
    KFold(n_splits=10, random_state=None, shuffle=False)

    accuracy=[]
    precision = []
    recall = []
    f1 = []
    
    temp = 0
    new_clf = LogisticRegression(random_state=0)
    
    for train_index, test_index in kf.split(X):
        #print("Train:", train_index, "Test:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        clf_report = classification_report(y_test, predictions, output_dict = True)
        accuracy.append(clf_report['accuracy'])
        precision.append(clf_report['weighted avg']['precision'])
        recall.append(clf_report['weighted avg']['recall'])
        f1.append(clf_report['weighted avg']['f1-score'])        
    print("Accuracy: {}, Precision: {}, Recall: {}, F1-score: {}".format(mean(accuracy),mean(precision),mean(recall),mean(f1)))
    if z is not None:
        predictions = new_clf.predict(z)
        return predictions
    else:
        return None
    

In [6]:
def voting(X,y,z =None):
    from warnings import simplefilter
    simplefilter(action='ignore', category=FutureWarning)
    lr = LogisticRegression()
    svc = SVC(kernel='poly',probability=True)
    dt = DecisionTreeClassifier()
    classifiers = [('lr',lr),('dt',dt),('svc',svc)]
    clf = VotingClassifier(estimators = classifiers, voting = 'hard')
    
    kf = KFold(n_splits=10)
    kf.get_n_splits(X)
    KFold(n_splits=10, random_state=None, shuffle=False)

    accuracy=[]
    precision = []
    recall = []
    f1 = []
    
    temp = 0
    new_clf = LogisticRegression(random_state=0)
    
    for train_index, test_index in kf.split(X):
        #print("Train:", train_index, "Test:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        clf_report = classification_report(y_test, predictions, output_dict = True)
        accuracy.append(clf_report['accuracy'])
        precision.append(clf_report['weighted avg']['precision'])
        recall.append(clf_report['weighted avg']['recall'])
        f1.append(clf_report['weighted avg']['f1-score'])        
    print("Accuracy: {}, Precision: {}, Recall: {}, F1-score: {}".format(mean(accuracy),mean(precision),mean(recall),mean(f1)))
    if z is not None:
        predictions = new_clf.predict(z)
        return predictions
    else:
        return None
    

In [7]:
def feedforward_net(X,y,z =None):
    np.random.seed(1337)

    nb_classes = 2

    batch_size = 64

    nb_epochs = 30
    
    kf = KFold(n_splits=2)
    kf.get_n_splits(X)
    KFold(n_splits=2, random_state=123, shuffle=True)

    accuracy=[]
    precision = []
    recall = []
    f1 = []
    
    temp = 0
    
    # 3 layer, Hidden layer size 1000, 500, 50 activation Relu, drop outs loss function: cross entropy batch size  = 64 
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = np_utils.to_categorical(y[train_index], nb_classes), y[test_index]
        model = Sequential()

        model.add(Dense(1000,input_shape=(10000,)))

        model.add(Activation('relu'))

        model.add(Dropout(0.5))

        model.add(Dense(500))

        model.add(Activation('relu'))

        model.add(Dropout(0.5))

        model.add(Dense(50))

        model.add(Activation('relu'))

        model.add(Dropout(0.5))

        model.add(Dense(nb_classes))

        model.add(Activation('softmax'))

        model.compile(loss='categorical_crossentropy', optimizer= keras.optimizers.Adam(lr=5e-5, beta_1=0.9, beta_2=0.999, amsgrad=False),metrics=['acc'])

        model.fit(X_train, y_train, batch_size=batch_size, validation_split=0.15 ,epochs=nb_epochs, shuffle=True)
        predictions = model.predict_classes(X_test,batch_size=batch_size)
        clf_report = classification_report(y_test, predictions, output_dict = True)
        accuracy.append(clf_report['accuracy'])
        precision.append(clf_report['weighted avg']['precision'])
        recall.append(clf_report['weighted avg']['recall'])
        f1.append(clf_report['weighted avg']['f1-score'])        
    print("Accuracy: {}, Precision: {}, Recall: {}, F1-score: {}".format(mean(accuracy),mean(precision),mean(recall),mean(f1)))
    if z is not None:
        predictions = model.predict_classes(z,batch_size=batch_size)
        return predictions
    else:
        return None
 
    

In [39]:
def convo_nn(X,y,z =None):
    
    nb_classes = 2
    
    kf = KFold(n_splits=2)
    kf.get_n_splits(X)
    KFold(n_splits=2, random_state=123, shuffle=True)

    accuracy=[]
    precision = []
    recall = []
    f1 = []

    X = X.toarray()
    X = np.expand_dims(X, axis=-1)
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test =  y[train_index], y[test_index]
        input = Input(batch_shape=(None, 10000, 1))
        drop20 = SpatialDropout1D(0.3)(input)
        conv2 = Conv1D(filters=128, kernel_size=3, activation='relu')(drop20)
        drop21 = Dropout(0.5)(conv2)
        conv22 = Conv1D(filters=64, kernel_size=3, activation='relu')(drop21)
        drop22 = Dropout(0.5)(conv22)
        pool2 = MaxPooling1D(pool_size=2)(drop22)
        flat2 = Flatten()(pool2)
        out = Dense(1, activation='sigmoid')(flat2)

        model = Model(input, out)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.summary()
        hist = model.fit(X_train, y_train, epochs=10, validation_split=0.15, shuffle=True, batch_size=64)
        predictions = model.predict(X_test, batch_size=64, verbose=1)
        for i in range(len(predictions)):
            if predictions[i] > 0.5:
                predictions[i] = 1
            else:
                predictions[i] = 0
        clf_report = classification_report(y_test, predictions, output_dict = True)
        accuracy.append(clf_report['accuracy'])
        precision.append(clf_report['weighted avg']['precision'])
        recall.append(clf_report['weighted avg']['recall'])
        f1.append(clf_report['weighted avg']['f1-score'])
    print("Accuracy: {}, Precision: {}, Recall: {}, F1-score: {}".format(mean(accuracy),mean(precision),mean(recall),mean(f1)))
    if z is not None:
        predictions = model.predict_classes(z,batch_size=64)
        return predictions
    else:
        return None


In [40]:
def LSTmemory(X,y,z =None):
    max_features = 10000
    embedding_dim = 10
    epochs = 5
    batch_size = 100
    nb_classes = 2
    
    kf = KFold(n_splits=2)
    kf.get_n_splits(X)
    KFold(n_splits=2, random_state=123, shuffle=True)

    accuracy=[]
    precision = []
    recall = []
    f1 = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = np_utils.to_categorical(y[train_index], nb_classes), y[test_index]

        model = Sequential()
        model.add(Embedding(max_features, embedding_dim, input_length = X_train.shape[1]))
        model.add(LSTM(100, dropout = 0.2, recurrent_dropout = 0.2))
        model.add(Dense(2, activation = 'softmax'))
        model.compile(loss = 'categorical_crossentropy', optimizer= keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False),metrics = ['accuracy'])
        print(model.summary())
        model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_split = 0.2, callbacks = [EarlyStopping(monitor = 'val_loss', patience = 3, min_delta = 0.00001)])
        predictions = model.predict_classes(X_test,batch_size=batch_size)
        clf_report = classification_report(y_test, predictions, output_dict = True)
        accuracy.append(clf_report['accuracy'])
        precision.append(clf_report['weighted avg']['precision'])
        recall.append(clf_report['weighted avg']['recall'])
        f1.append(clf_report['weighted avg']['f1-score'])        
    print("Accuracy: {}, Precision: {}, Recall: {}, F1-score: {}".format(mean(accuracy),mean(precision),mean(recall),mean(f1)))
    if z is not None:
        predictions = model.predict_classes(z,batch_size=batch_size)
        return predictions
    else:
        return None  
    
    
    
    
    

In [41]:
def extract_features(training_data,type="binary"):
    """Extract features using different methods"""
    
    
    if "binary" in type:
        
        # BINARY FEATURE REPRESENTATION
        cv= CountVectorizer(binary=True, min_df=2, ngram_range=(1, 2), stop_words='english', max_features= 10000,strip_accents='unicode')
        cv.fit_transform(training_data["content"].values)

        train_feature_set=cv.transform(training_data["content"].values)
        
        return train_feature_set
  
    elif "counts" in type:
        
        # COUNT BASED FEATURE REPRESENTATION
        cv= CountVectorizer(binary=False, min_df=2, ngram_range=(1, 2), stop_words='english', max_features= 10000,strip_accents='unicode')
        cv.fit_transform(training_data["content"].values)
        train_feature_set=cv.transform(training_data["content"].values)
        
        return train_feature_set
    
    else:    
        
        # TF-IDF BASED FEATURE REPRESENTATION
        cv=TfidfVectorizer(min_df=2, ngram_range=(1, 2), stop_words='english', max_features= 10000,strip_accents='unicode', norm='l2')
        
        cv.fit_transform(training_data["content"].values)

        train_feature_set=cv.transform(training_data["content"].values)
        
        return train_feature_set

In [42]:
def machine_learning_models(X,y,model,text_length,type,z=None):
    if text_length is True:
        X = X.apply(preprocess_paragraph, axis = 1)
        X = extract_features(X,type=type)
        if z is not None:
            z = z.apply(preprocess_paragraph, axis = 1)
            z = extract_features(z,type=type)
        else:
            pass
    else:
        X = X.apply(preprocess_full_text, axis = 1)
        X = extract_features(X,type=type)
        if z is not None:
            z = z.apply(preprocess_full_text, axis = 1)
            z = extract_features(z,type=type)
        else:
            pass

    if model == 'svm':
        z = support_vm(X,y,z)
    elif model == 'rf':
        z = random_forest(X,y,z)
    elif model == 'logit':
        z = logit(X,y,z)
    elif model == 'xgb':
        z = xgboost(X,y,z)
    elif model == 'cnn':
        z = convo_nn(X,y,z)
    elif model == 'lstm':
        z = LSTmemory(X,y,z)
    elif model == 'dct':
        z = dctree(X,y,z)
    elif model == 'voting':
        z = voting(X,y,z)
    else:
        z = feedforward_net(X,y,z)
        
    return z

In [43]:
path = 'MASTER_TRAINING.xlsx'
X = pd.read_excel(path)
columns = ['content', 'licensing_agreement']
X = X[columns]
X = X.dropna(subset = ['licensing_agreement'])
y = X['licensing_agreement']


In [13]:
z = pd.read_csv('2000.csv')

prediction

In [22]:
machine_learning_models(X,y,'voting', True, type='tfidf')
machine_learning_models(X,y,'voting', True, type='binary')
machine_learning_models(X,y,'voting', True, type='count')
machine_learning_models(X,y,'voting', False, type='tfidf')
machine_learning_models(X,y,'voting', False, type='count')
machine_learning_models(X,y,'voting', False, type='binary')

Accuracy: 0.78500500607881, Precision: 0.8106178057694198, Recall: 0.78500500607881, F1-score: 0.777103829942487
Accuracy: 0.7714742902095402, Precision: 0.7969097425243691, Recall: 0.7714742902095402, F1-score: 0.7625165822356037
Accuracy: 0.7854340985482371, Precision: 0.8125116939386764, Recall: 0.7854340985482371, F1-score: 0.7773849006361659
Accuracy: 0.7964152899949939, Precision: 0.8168085676317636, Recall: 0.7964152899949939, F1-score: 0.7906343724110119
Accuracy: 0.7955731960237431, Precision: 0.8148391180884533, Recall: 0.7955731960237431, F1-score: 0.7898419651662243
Accuracy: 0.7799184724308088, Precision: 0.8024805117071274, Recall: 0.7799184724308088, F1-score: 0.7724652164887211


In [23]:
machine_learning_models(X,y,'svm',True,type="binary")
machine_learning_models(X,y,'svm',True,type="counts")
machine_learning_models(X,y,'svm',True,type="tfidf")
machine_learning_models(X,y,'svm',False,type="binary")
machine_learning_models(X,y,'svm',False,type="counts")
machine_learning_models(X,y,'svm',False,type="tfidf")

Accuracy: 0.790908603304012, Precision: 0.7992307625065564, Recall: 0.790908603304012, F1-score: 0.7891153323216827
Accuracy: 0.7570746620896803, Precision: 0.7708721344808916, Recall: 0.7570746620896803, F1-score: 0.7539331748316717
Accuracy: 0.813748837874562, Precision: 0.8214044370075727, Recall: 0.813748837874562, F1-score: 0.8118173579842061
Accuracy: 0.7718801401702067, Precision: 0.7782662332729737, Recall: 0.7718801401702067, F1-score: 0.7710506282995995
Accuracy: 0.7769648859329186, Precision: 0.782532198645103, Recall: 0.7769648859329186, F1-score: 0.7763827055514145
Accuracy: 0.8116570120861045, Precision: 0.8142204905378789, Recall: 0.8116570120861045, F1-score: 0.8115667961740902


In [37]:
machine_learning_models(X,y,'rf',True,type="binary")
machine_learning_models(X,y,'rf',True,type="counts")
machine_learning_models(X,y,'rf',True,type="tfidf")
machine_learning_models(X,y,'rf',False,type="binary")
machine_learning_models(X,y,'rf',False,type="counts")
machine_learning_models(X,y,'rf',False,type="tfidf")

Accuracy: 0.7799399270542802, Precision: 0.7887309000202752, Recall: 0.7799399270542802, F1-score: 0.7765061452592132
Accuracy: 0.7854591289422871, Precision: 0.7956111185610738, Recall: 0.7854591289422871, F1-score: 0.7823906529311617
Accuracy: 0.7651702066795395, Precision: 0.7764758605150585, Recall: 0.7651702066795395, F1-score: 0.7616400505679016
Accuracy: 0.7854305227776586, Precision: 0.7949019195896372, Recall: 0.7854305227776586, F1-score: 0.782900736340923
Accuracy: 0.7850032181935207, Precision: 0.7964923812672423, Recall: 0.7850032181935207, F1-score: 0.7819347354499013
Accuracy: 0.7689801902309947, Precision: 0.7816960411083437, Recall: 0.7689801902309947, F1-score: 0.7646248873300331


In [38]:
machine_learning_models(X,y,'logit',True,type="binary")
machine_learning_models(X,y,'logit',True,type="counts")
machine_learning_models(X,y,'logit',True,type="tfidf")
machine_learning_models(X,y,'logit',False,type="binary")
machine_learning_models(X,y,'logit',False,type="counts")
machine_learning_models(X,y,'logit',False,type="tfidf")

Accuracy: 0.8183884002002432, Precision: 0.8260482828611077, Recall: 0.8183884002002432, F1-score: 0.816466729243373
Accuracy: 0.8128817135092612, Precision: 0.8213530393768742, Recall: 0.8128817135092612, F1-score: 0.8109287561381117
Accuracy: 0.823887935350068, Precision: 0.8328068869525717, Recall: 0.823887935350068, F1-score: 0.8219948067838223
Accuracy: 0.8170975470213832, Precision: 0.8216041955498405, Recall: 0.8170975470213832, F1-score: 0.8161467315603862
Accuracy: 0.8065400843881857, Precision: 0.8116872686650721, Recall: 0.8065400843881857, F1-score: 0.805821163701258
Accuracy: 0.8302224129299864, Precision: 0.8337294455875043, Recall: 0.8302224129299864, F1-score: 0.8296743581190806


In [39]:
machine_learning_models(X,y,'xgb',True,type="binary")
machine_learning_models(X,y,'xgb',True,type="counts")
machine_learning_models(X,y,'xgb',True,type="tfidf")
machine_learning_models(X,y,'xgb',False,type="binary")
machine_learning_models(X,y,'xgb',False,type="counts")
machine_learning_models(X,y,'xgb',False,type="tfidf")

Accuracy: 0.7731656296931989, Precision: 0.7853375211237872, Recall: 0.7731656296931989, F1-score: 0.7703773425555043
Accuracy: 0.7799506543660158, Precision: 0.7910959578407073, Recall: 0.7799506543660158, F1-score: 0.7768099659240104
Accuracy: 0.787534863763141, Precision: 0.7971109256330073, Recall: 0.787534863763141, F1-score: 0.7842189264790582
Accuracy: 0.7773814632053208, Precision: 0.7881355875454753, Recall: 0.7773814632053208, F1-score: 0.7755774837920097
Accuracy: 0.7812093256096688, Precision: 0.7910415349552208, Recall: 0.7812093256096688, F1-score: 0.7791019420040333
Accuracy: 0.7866802545948652, Precision: 0.7948739036116249, Recall: 0.7866802545948652, F1-score: 0.784344186595227


In [41]:
machine_learning_models(X,y,'feed_forward_nn',True,type="tfidf" )

Train on 1005 samples, validate on 178 samples
Epoch 1/30
1005/1005 [==============================] - 11s 11ms/step - loss: 0.6935 - acc: 0.4995 - val_loss: 0.6913 - val_acc: 0.5730
Epoch 2/30
1005/1005 [==============================] - 8s 8ms/step - loss: 0.6900 - acc: 0.5751 - val_loss: 0.6871 - val_acc: 0.6292
Epoch 3/30
1005/1005 [==============================] - 6s 6ms/step - loss: 0.6856 - acc: 0.6209 - val_loss: 0.6818 - val_acc: 0.6292
Epoch 4/30
1005/1005 [==============================] - 6s 6ms/step - loss: 0.6811 - acc: 0.6428 - val_loss: 0.6745 - val_acc: 0.6292
Epoch 5/30
1005/1005 [==============================] - 6s 6ms/step - loss: 0.6725 - acc: 0.6697 - val_loss: 0.6646 - val_acc: 0.6348
Epoch 6/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6640 - acc: 0.6896 - val_loss: 0.6510 - val_acc: 0.6517
Epoch 7/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6476 - acc: 0.7234 - val_loss: 0.6318 - val_acc: 0.6798
Epoch 8/30
10

In [42]:
machine_learning_models(X,y,'feed_forward_nn',True,type='binary')

Train on 1005 samples, validate on 178 samples
Epoch 1/30
1005/1005 [==============================] - 8s 8ms/step - loss: 0.7312 - acc: 0.4935 - val_loss: 0.6367 - val_acc: 0.6966
Epoch 2/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6682 - acc: 0.6030 - val_loss: 0.6026 - val_acc: 0.6854
Epoch 3/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6406 - acc: 0.6259 - val_loss: 0.5645 - val_acc: 0.7472
Epoch 4/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.5927 - acc: 0.6975 - val_loss: 0.5184 - val_acc: 0.8427
Epoch 5/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.5566 - acc: 0.7353 - val_loss: 0.4724 - val_acc: 0.8652
Epoch 6/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.5342 - acc: 0.7502 - val_loss: 0.4308 - val_acc: 0.8764
Epoch 7/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.4956 - acc: 0.7791 - val_loss: 0.3983 - val_acc: 0.8820
Epoch 8/30
1005

In [43]:
machine_learning_models(X,y,'feed_forward_nn',True,type='counts')

Train on 1005 samples, validate on 178 samples
Epoch 1/30
1005/1005 [==============================] - 7s 7ms/step - loss: 1.2108 - acc: 0.4915 - val_loss: 0.6360 - val_acc: 0.5506
Epoch 2/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.9232 - acc: 0.5363 - val_loss: 0.6058 - val_acc: 0.5618
Epoch 3/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.8030 - acc: 0.5572 - val_loss: 0.5522 - val_acc: 0.6236
Epoch 4/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.7097 - acc: 0.6299 - val_loss: 0.5186 - val_acc: 0.6573
Epoch 5/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.7132 - acc: 0.6348 - val_loss: 0.4958 - val_acc: 0.6854
Epoch 6/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6191 - acc: 0.6796 - val_loss: 0.4818 - val_acc: 0.7022
Epoch 7/30
1005/1005 [==============================] - 6s 6ms/step - loss: 0.5912 - acc: 0.7184 - val_loss: 0.4685 - val_acc: 0.7584
Epoch 8/30
1005

In [44]:
machine_learning_models(X,y,'feed_forward_nn',False,type='counts')

Train on 1005 samples, validate on 178 samples
Epoch 1/30
1005/1005 [==============================] - 6s 6ms/step - loss: 1.0961 - acc: 0.5065 - val_loss: 0.5711 - val_acc: 0.6011
Epoch 2/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.8883 - acc: 0.5473 - val_loss: 0.5425 - val_acc: 0.6180
Epoch 3/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.7718 - acc: 0.5920 - val_loss: 0.4997 - val_acc: 0.6573
Epoch 4/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.7005 - acc: 0.6318 - val_loss: 0.4740 - val_acc: 0.7191
Epoch 5/30
1005/1005 [==============================] - 6s 6ms/step - loss: 0.6485 - acc: 0.6706 - val_loss: 0.4613 - val_acc: 0.7584
Epoch 6/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6410 - acc: 0.6816 - val_loss: 0.4514 - val_acc: 0.7640
Epoch 7/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.5696 - acc: 0.7045 - val_loss: 0.4400 - val_acc: 0.7753
Epoch 8/30
1005

In [45]:
machine_learning_models(X,y,'feed_forward_nn',False,type='binary')

Train on 1005 samples, validate on 178 samples
Epoch 1/30
1005/1005 [==============================] - 6s 6ms/step - loss: 0.7447 - acc: 0.4995 - val_loss: 0.6388 - val_acc: 0.6461
Epoch 2/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6831 - acc: 0.5950 - val_loss: 0.5889 - val_acc: 0.6910
Epoch 3/30
1005/1005 [==============================] - 6s 6ms/step - loss: 0.6262 - acc: 0.6328 - val_loss: 0.5474 - val_acc: 0.7472
Epoch 4/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.5951 - acc: 0.6667 - val_loss: 0.5085 - val_acc: 0.8034
Epoch 5/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.5613 - acc: 0.7134 - val_loss: 0.4734 - val_acc: 0.8371
Epoch 6/30
1005/1005 [==============================] - 6s 6ms/step - loss: 0.5329 - acc: 0.7522 - val_loss: 0.4414 - val_acc: 0.8427
Epoch 7/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.4874 - acc: 0.7881 - val_loss: 0.4117 - val_acc: 0.8539
Epoch 8/30
1005

In [46]:
machine_learning_models(X,y,'feed_forward_nn',False,type='tfidf')

Train on 1005 samples, validate on 178 samples
Epoch 1/30
1005/1005 [==============================] - 6s 6ms/step - loss: 0.6928 - acc: 0.5174 - val_loss: 0.6889 - val_acc: 0.7022
Epoch 2/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6860 - acc: 0.6030 - val_loss: 0.6823 - val_acc: 0.6236
Epoch 3/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6821 - acc: 0.6109 - val_loss: 0.6751 - val_acc: 0.6348
Epoch 4/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6766 - acc: 0.6179 - val_loss: 0.6654 - val_acc: 0.6292
Epoch 5/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6635 - acc: 0.6488 - val_loss: 0.6525 - val_acc: 0.6404
Epoch 6/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6530 - acc: 0.6697 - val_loss: 0.6360 - val_acc: 0.6517
Epoch 7/30
1005/1005 [==============================] - 5s 5ms/step - loss: 0.6349 - acc: 0.6925 - val_loss: 0.6141 - val_acc: 0.6573
Epoch 8/30
1005

In [44]:
machine_learning_models(X,y,'cnn',True,type="tfidf" )

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 10000, 1)          0         
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 10000, 1)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 9998, 128)         512       
_________________________________________________________________
dropout_9 (Dropout)          (None, 9998, 128)         0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 9996, 64)          24640     
_________________________________________________________________
dropout_10 (Dropout)         (None, 9996, 64)          0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 4998, 64)          0   

In [45]:
### FROM HERE

In [126]:
machine_learning_models(X,y,'cnn',True,type="binary" )

Train on 1005 samples, validate on 178 samples
Epoch 1/30
1005/1005 [==============================] - 9s 9ms/step - loss: 0.7378 - acc: 0.5224 - val_loss: 0.6414 - val_acc: 0.6854
Epoch 2/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6764 - acc: 0.5841 - val_loss: 0.6079 - val_acc: 0.8258
Epoch 3/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6517 - acc: 0.6179 - val_loss: 0.5778 - val_acc: 0.8539
Epoch 4/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6299 - acc: 0.6458 - val_loss: 0.5499 - val_acc: 0.8427
Epoch 5/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.5939 - acc: 0.6687 - val_loss: 0.5164 - val_acc: 0.8596
Epoch 6/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.5712 - acc: 0.6915 - val_loss: 0.4792 - val_acc: 0.8764
Epoch 7/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.5544 - acc: 0.7363 - val_loss: 0.4493 - val_acc: 0.8876
Epoch 8/30
1005

In [127]:
machine_learning_models(X,y,'cnn',True,type="counts" )

Train on 1005 samples, validate on 178 samples
Epoch 1/30
1005/1005 [==============================] - 9s 9ms/step - loss: 0.9844 - acc: 0.5284 - val_loss: 0.6318 - val_acc: 0.5787
Epoch 2/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.8529 - acc: 0.5602 - val_loss: 0.5953 - val_acc: 0.6348
Epoch 3/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.7856 - acc: 0.5602 - val_loss: 0.5781 - val_acc: 0.6966
Epoch 4/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.7222 - acc: 0.5990 - val_loss: 0.5716 - val_acc: 0.7809
Epoch 5/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6873 - acc: 0.6249 - val_loss: 0.5620 - val_acc: 0.7753
Epoch 6/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6810 - acc: 0.6408 - val_loss: 0.5494 - val_acc: 0.7921
Epoch 7/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6760 - acc: 0.6647 - val_loss: 0.5366 - val_acc: 0.8202
Epoch 8/30
1005

In [128]:
machine_learning_models(X,y,'cnn',False,type="counts" )

Train on 1005 samples, validate on 178 samples
Epoch 1/30
1005/1005 [==============================] - 9s 9ms/step - loss: 0.9786 - acc: 0.5154 - val_loss: 0.6307 - val_acc: 0.6180
Epoch 2/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.8354 - acc: 0.5502 - val_loss: 0.5886 - val_acc: 0.7135
Epoch 3/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.7559 - acc: 0.5891 - val_loss: 0.5610 - val_acc: 0.7584
Epoch 4/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.7241 - acc: 0.6050 - val_loss: 0.5497 - val_acc: 0.7697
Epoch 5/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6817 - acc: 0.6308 - val_loss: 0.5369 - val_acc: 0.7865
Epoch 6/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6624 - acc: 0.6527 - val_loss: 0.5211 - val_acc: 0.7809
Epoch 7/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6527 - acc: 0.6448 - val_loss: 0.5085 - val_acc: 0.8034
Epoch 8/30
1005

In [130]:
machine_learning_models(X,y,'cnn',False,type="tfidf" )

Train on 1005 samples, validate on 178 samples
Epoch 1/30
1005/1005 [==============================] - 10s 10ms/step - loss: 0.9786 - acc: 0.5154 - val_loss: 0.6307 - val_acc: 0.6180
Epoch 2/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.8354 - acc: 0.5502 - val_loss: 0.5886 - val_acc: 0.7135
Epoch 3/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.7559 - acc: 0.5891 - val_loss: 0.5610 - val_acc: 0.7584
Epoch 4/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.7241 - acc: 0.6050 - val_loss: 0.5497 - val_acc: 0.7697
Epoch 5/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6817 - acc: 0.6308 - val_loss: 0.5369 - val_acc: 0.7865
Epoch 6/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6624 - acc: 0.6527 - val_loss: 0.5211 - val_acc: 0.7809
Epoch 7/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6527 - acc: 0.6448 - val_loss: 0.5085 - val_acc: 0.8034
Epoch 8/30
10

In [129]:
machine_learning_models(X,y,'cnn',False,type="binary" )

Train on 1005 samples, validate on 178 samples
Epoch 1/30
1005/1005 [==============================] - 9s 9ms/step - loss: 0.9786 - acc: 0.5154 - val_loss: 0.6307 - val_acc: 0.6180
Epoch 2/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.8354 - acc: 0.5502 - val_loss: 0.5886 - val_acc: 0.7135
Epoch 3/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.7559 - acc: 0.5891 - val_loss: 0.5610 - val_acc: 0.7584
Epoch 4/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.7241 - acc: 0.6050 - val_loss: 0.5497 - val_acc: 0.7697
Epoch 5/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6817 - acc: 0.6308 - val_loss: 0.5369 - val_acc: 0.7865
Epoch 6/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6624 - acc: 0.6527 - val_loss: 0.5211 - val_acc: 0.7809
Epoch 7/30
1005/1005 [==============================] - 1s 1ms/step - loss: 0.6527 - acc: 0.6448 - val_loss: 0.5085 - val_acc: 0.8034
Epoch 8/30
1005

In [93]:
machine_learning_models(X,y,'lstm',True,type='tfidf')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 5000, 10)          500000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dense_69 (Dense)             (None, 2)                 202       
Total params: 544,602
Trainable params: 544,602
Non-trainable params: 0
_________________________________________________________________
None
Train on 1514 samples, validate on 379 samples
Epoch 1/10
1514/1514 [==============================] - 406s 268ms/step - loss: 0.6931 - acc: 0.5119 - val_loss: 0.6927 - val_acc: 0.5620
Epoch 2/10
1514/1514 [==============================] - 394s 260ms/step - loss: 0.6929 - acc: 0.5310 - val_loss: 0.6923 - val_acc: 0.5620
Epoch 3/10
1514/1514 [==============================] - 398s 263ms/step 

/Users/sunwoo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 5000, 10)          500000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dense_70 (Dense)             (None, 2)                 202       
Total params: 544,602
Trainable params: 544,602
Non-trainable params: 0
_________________________________________________________________
None
Train on 1514 samples, validate on 379 samples
Epoch 1/10
1514/1514 [==============================] - 418s 276ms/step - loss: 0.6922 - acc: 0.5403 - val_loss: 0.6914 - val_acc: 0.5620
Epoch 2/10
1514/1514 [==============================] - 398s 263ms/step - loss: 0.6915 - acc: 0.5469 - val_loss: 0.6905 - val_acc: 0.5620
Epoch 3/10
1514/1514 [==============================] - 398s 263ms/step 

/Users/sunwoo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 5000, 10)          500000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dense_71 (Dense)             (None, 2)                 202       
Total params: 544,602
Trainable params: 544,602
Non-trainable params: 0
_________________________________________________________________
None
Train on 1515 samples, validate on 379 samples
Epoch 1/10
1515/1515 [==============================] - 429s 283ms/step - loss: 0.6932 - acc: 0.5030 - val_loss: 0.6928 - val_acc: 0.5620
Epoch 2/10
1515/1515 [==============================] - 412s 272ms/step - loss: 0.6926 - acc: 0.5287 - val_loss: 0.6920 - val_acc: 0.5620
Epoch 3/10
1515/1515 [==============================] - 400s 264ms/step 

/Users/sunwoo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 5000, 10)          500000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dense_72 (Dense)             (None, 2)                 202       
Total params: 544,602
Trainable params: 544,602
Non-trainable params: 0
_________________________________________________________________
None
Train on 1515 samples, validate on 379 samples
Epoch 1/10
1515/1515 [==============================] - 400s 264ms/step - loss: 0.6926 - acc: 0.5248 - val_loss: 0.6918 - val_acc: 0.5620
Epoch 2/10
1515/1515 [==============================] - 399s 264ms/step - loss: 0.6923 - acc: 0.5380 - val_loss: 0.6910 - val_acc: 0.5620
Epoch 3/10
1515/1515 [==============================] - 393s 259ms/step 

/Users/sunwoo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 5000, 10)          500000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dense_73 (Dense)             (None, 2)                 202       
Total params: 544,602
Trainable params: 544,602
Non-trainable params: 0
_________________________________________________________________
None
Train on 1515 samples, validate on 379 samples
Epoch 1/10
1515/1515 [==============================] - 425s 281ms/step - loss: 0.6927 - acc: 0.5261 - val_loss: 0.6923 - val_acc: 0.5383
Epoch 2/10
1515/1515 [==============================] - 393s 259ms/step - loss: 0.6925 - acc: 0.5267 - val_loss: 0.6920 - val_acc: 0.5383
Epoch 3/10
1515/1515 [==============================] - 393s 259ms/step 

/Users/sunwoo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [132]:
machine_learning_models(X,y,'lstm',True,type='binary')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 5000, 10)          500000    
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dense_152 (Dense)            (None, 2)                 202       
Total params: 544,602
Trainable params: 544,602
Non-trainable params: 0
_________________________________________________________________
None
Train on 946 samples, validate on 237 samples
Epoch 1/5
946/946 [==============================] - 263s 279ms/step - loss: 0.6905 - acc: 0.5518 - val_loss: 0.6791 - val_acc: 0.5992
Epoch 2/5
946/946 [==============================] - 258s 272ms/step - loss: 0.6896 - acc: 0.5444 - val_loss: 0.6775 - val_acc: 0.5992
Epoch 3/5
946/946 [==============================] - 269s 285ms/step - loss: 0.

/Users/sunwoo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 5000, 10)          500000    
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               44400     
_________________________________________________________________
dense_153 (Dense)            (None, 2)                 202       
Total params: 544,602
Trainable params: 544,602
Non-trainable params: 0
_________________________________________________________________
None
Train on 947 samples, validate on 237 samples
Epoch 1/5
947/947 [==============================] - 257s 271ms/step - loss: 0.6929 - acc: 0.5121 - val_loss: 0.6926 - val_acc: 0.5190
Epoch 2/5
947/947 [==============================] - 245s 259ms/step - loss: 0.6922 - acc: 0.5227 - val_loss: 0.6927 - val_acc: 0.5190
Epoch 3/5
947/947 [==============================] - 246s 260ms/step - loss: 0.

/Users/sunwoo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
machine_learning_models(X,y,'lstm',True,type="counts" )

In [ ]:
machine_learning_models(X,y,'lstm',False,type="counts" )

In [ ]:
machine_learning_models(X,y,'lstm',False,type="binary" )

In [ ]:
machine_learning_models(X,y,'lstm',False,type="tfidf" )